In [ ]:
Пример решения задачи множественной регрессии с помощью Python

https://habr.com/post/206306/

Введение

Добрый день, уважаемые читатели.
В прошлых статьях, на практических примерах, мной были показаны способы решения задач классификации (задача кредитного скоринга (http://habrahabr.ru/post/204500/)) и основ анализа текстовой информации (задача о паспортах). Сегодня же мне бы хотелось коснуться другого класса задач, а именно восстановления регрессии. Задачи данного класса, как правило, используются при прогнозировании.
Для примера решения задачи прогнозирования, я взял набор данных Energy efficiency из крупнейшего репозитория UCI. В качестве инструментов по традиции будем использовать Python c аналитическими пакетами pandas и scikit-learn.

Описание набора данных и постановка задачи

Дан набор данных, котором описаны следующие атрибуты помещения:
Поле	Описание							Тип
X1		Относительная компактность			FLOAT
X2		Площадь								FLOAT
X3		Площадь стены						FLOAT
X4		Площадь потолка						FLOAT
X5		Общая высота						FLOAT
X6		Ориентация							INT
X7		Площадь остекления					FLOAT
X8		Распределенная площадь остекления	INT
y1		Нагрузка при обогреве				FLOAT
y2		Нагрузка при охлаждении				FLOAT

Предварительный анализ данных

Для начала загрузим наши данные и посмотрим на них:


In [2]:
from pandas import read_csv, DataFrame
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split



In [9]:
dataset = read_csv('datasets\\ENB2012_data.csv',';')
dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,Unnamed: 10,Unnamed: 11
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33,NaN,NaN
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33,NaN,NaN
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33,NaN,NaN
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33,NaN,NaN
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28,NaN,NaN


Теперь давайте посмотрим не связаны ли между собой какие-либо атрибуты. Сделать это можно рассчитав коэффициенты корреляции для всех столбцов. Как это сделать было описано в предыдущей статье (http://habrahabr.ru/post/204500/):

In [10]:
dataset.corr()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,Unnamed: 10,Unnamed: 11
X1,1.000000e+00,-9.919015e-01,-2.037817e-01,-8.688234e-01,8.277473e-01,0.000000,1.283986e-17,1.764620e-17,0.622272,0.634339,NaN,NaN
X2,-9.919015e-01,1.000000e+00,1.955016e-01,8.807195e-01,-8.581477e-01,0.000000,1.318356e-16,-3.558613e-16,-0.658120,-0.672999,NaN,NaN
X3,-2.037817e-01,1.955016e-01,1.000000e+00,-2.923165e-01,2.809757e-01,0.000000,-7.969726e-19,0.000000e+00,0.455671,0.427117,NaN,NaN
X4,-8.688234e-01,8.807195e-01,-2.923165e-01,1.000000e+00,-9.725122e-01,0.000000,-1.381805e-16,-1.079129e-16,-0.861828,-0.862547,NaN,NaN
X5,8.277473e-01,-8.581477e-01,2.809757e-01,-9.725122e-01,1.000000e+00,0.000000,1.861418e-18,0.000000e+00,0.889431,0.895785,NaN,NaN
X6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,-0.002587,0.014290,NaN,NaN
X7,1.283986e-17,1.318356e-16,-7.969726e-19,-1.381805e-16,1.861418e-18,0.000000,1.000000e+00,2.129642e-01,0.269841,0.207505,NaN,NaN
X8,1.764620e-17,-3.558613e-16,0.000000e+00,-1.079129e-16,0.000000e+00,0.000000,2.129642e-01,1.000000e+00,0.087368,0.050525,NaN,NaN
Y1,6.222722e-01,-6.581202e-01,4.556712e-01,-8.618283e-01,8.894307e-01,-0.002587,2.698410e-01,8.736759e-02,1.000000,0.975862,NaN,NaN
Y2,6.343391e-01,-6.729989e-01,4.271170e-01,-8.625466e-01,8.957852e-01,0.014290,2.075050e-01,5.052512e-02,0.975862,1.000000,NaN,NaN


Как можно заметить из нашей матрицы, коррелируют между собой следующие столбы (Значение коэффициента корреляции больше 95%):
y1 --> y2
x1 --> x2
x4 --> x5

Теперь давайте выберем, какие столбцы их наших пар мы можем убрать из нашей выборки. Для этого, в каждой паре, выберем столбцы, которые в большей степени оказывают влияние на прогнозные значения Y1 и Y2 и оставим их, а остальные удалим.
Как можно заметить и матрицы с коэффициентами корреляции на y1,y2 больше значения оказывают X2 и X5, нежели X1 и X4, таким образом мы можем последние столбцы мы можем удалить.

In [11]:
dataset = dataset.drop(['X1','X4'], axis=1)
dataset.head()

,X2,X3,X5,X6,X7,X8,Y1,Y2,Unnamed: 10,Unnamed: 11
0,514.5,294.0,7.0,2.0,0.0,0.0,15.55,21.33,NaN,NaN
1,514.5,294.0,7.0,3.0,0.0,0.0,15.55,21.33,NaN,NaN
2,514.5,294.0,7.0,4.0,0.0,0.0,15.55,21.33,NaN,NaN
3,514.5,294.0,7.0,5.0,0.0,0.0,15.55,21.33,NaN,NaN
4,563.5,318.5,7.0,2.0,0.0,0.0,20.84,28.28,NaN,NaN
